# CFM 301 Data Assignment 1 Q4

Jeongseop Yi (j22yi / 20727267)

### Q4a

In [141]:
import pandas as pd
from pandasql import sqldf
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn import linear_model
import statsmodels.api as sm

In [142]:
data = pd.read_sas("../csv/q4a.sas7bdat", encoding='latin-1')
data.columns = map(str.lower, data.columns)

data["date"] = pd.to_datetime(data["date"], format='%Y%m%d')
data["yyyymm"] = data.date.dt.year*100 + data.date.dt.month
data["monthid"] = (data.date.dt.year-2005)*12 + data.date.dt.month

bhholding_data = pd.read_excel("../BHHoldings_202209.xlsx", sheet_name="BH_cleaned_permno")
bhholding_data.columns = map(str.lower, bhholding_data.columns)

In [143]:
print(data)
data = pd.merge(data, bhholding_data, on="permno", how='left')

       permno       date ticker       ret    vwretd  yyyymm  monthid
0     10933.0 2005-01-31    MKL -0.063187 -0.026546  200501        1
1     10933.0 2005-02-28    MKL  0.048387  0.022646  200502        2
2     10933.0 2005-03-31    MKL -0.034378 -0.016944  200503        3
3     10933.0 2005-04-29    MKL -0.006112 -0.025206  200504        4
4     10933.0 2005-05-31    MKL -0.003935  0.037954  200505        5
...       ...        ...    ...       ...       ...     ...      ...
7666  91937.0 2021-08-31   TMUS -0.048604  0.027147  202108      200
7667  91937.0 2021-09-30   TMUS -0.067581 -0.042243  202109      201
7668  91937.0 2021-10-29   TMUS -0.099640  0.064657  202110      202
7669  91937.0 2021-11-30   TMUS -0.054073 -0.018347  202111      203
7670  91937.0 2021-12-31   TMUS  0.065895  0.033345  202112      204

[7671 rows x 7 columns]


In [144]:
with pd.ExcelWriter("../xlsx/CA1_data.xlsx", mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
    data.to_excel(writer, sheet_name='q4a')

### Q4b

In [145]:
bh_mat = {'monthid':[], 'yyyymm':[], 'ret': []}

for mid in range(1, max(data['monthid']) + 1):
    month_data = pd.DataFrame(data.query('monthid == ' + str(mid)))
    bh_mat['monthid'].append(mid)
    bh_mat['yyyymm'].append(month_data['yyyymm'].iloc[0])

    weight_sum = month_data['portfolio_weight'].sum()
    month_data['weight_adj'] = month_data['portfolio_weight'].div(weight_sum).copy()
    month_data['ret_adj'] = month_data.apply(lambda row: (row['ret'] * row['weight_adj']), axis=1)
    bh_mat['ret'].append(month_data['ret_adj'].sum())

bh_mat = pd.DataFrame(bh_mat)

In [146]:
print(bh_mat)
print(bh_mat["ret"].std())

     monthid  yyyymm       ret
0          1  200501  0.074492
1          2  200502  0.098175
2          3  200503 -0.051562
3          4  200504 -0.063052
4          5  200505  0.069344
..       ...     ...       ...
199      200  202108  0.022521
200      201  202109 -0.025439
201      202  202110  0.068779
202      203  202111  0.005195
203      204  202112  0.059671

[204 rows x 3 columns]
0.06100240898981602


In [147]:
with pd.ExcelWriter("../xlsx/CA1_data.xlsx", mode='a', engine="openpyxl", if_sheet_exists='replace') as writer:
    bh_mat.to_excel(writer, sheet_name="q4b")

### Q4c

In [148]:
ff_factor = pd.read_csv("../csv/ff_factors.csv")
ff_factor.columns = map(str.lower, ff_factor.columns)

ff_factor['dateff'] = ff_factor["dateff"] = pd.to_datetime(ff_factor["dateff"], format='%Y%m%d')
ff_factor['yyyymm'] = ff_factor["yyyymm"] = ff_factor.dateff.dt.year*100 + ff_factor.dateff.dt.month

bh_mat = pd.merge(bh_mat, ff_factor, on="yyyymm", how='left')

In [149]:
bh_mat['xret'] = bh_mat['ret'] - bh_mat['rf']

x = sm.add_constant(bh_mat['mktrf'])
model = sm.OLS(bh_mat['xret'], x).fit()

model.params

const    0.007068
mktrf    1.187101
dtype: float64

In [150]:
explanatory_variates = bh_mat[["mktrf", "smb", "hml"]]

k = sm.add_constant(explanatory_variates)
model = sm.OLS(bh_mat['xret'], k).fit()

model.params

const    0.006905
mktrf    1.227393
smb     -0.224855
hml      0.023264
dtype: float64